# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

In [9]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://f94b3bff623f:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [10]:
from pyspark.sql.functions import get_json_object
from pyspark.sql.functions import from_json
from santiagomdeo.spark_utils import SparkUtils


agencies_schema = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
agencies_df = spark.read.option("header", True).schema(agencies_schema).csv("/opt/spark/work-dir/data/car_service/agencies")
agencies_df2 = agencies_df.withColumn("agency_name", get_json_object(agencies_df.agency_info, '$.agency_name'))
agencies_df3 = agencies_df2.withColumn("city", get_json_object(agencies_df2.agency_info, '$.city'))
agencies_df4 = agencies_df3.drop("agency_info")


int
string


In [11]:
brands_schema = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])
brands_df = spark.read.option("header", True).schema(brands_schema).csv("/opt/spark/work-dir/data/car_service/brands")
brands_df2 = brands_df.withColumn("brand_name", get_json_object(brands_df.brand_info, '$.brand_name'))
brands_df3 = brands_df2.withColumn("country", get_json_object(brands_df2.brand_info, '$.country'))
brands_df4 = brands_df3.drop("brand_info")

int
string


In [12]:
cars_schema = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
cars_df = spark.read.option("header", True).schema(cars_schema).csv("/opt/spark/work-dir/data/car_service/cars")
cars_df2 = cars_df.withColumn("car_name", get_json_object(cars_df.car_info, '$.car_name'))
cars_df3 = cars_df2.withColumn("brand_id", get_json_object(cars_df2.car_info, '$.brand_id'))
cars_df4 = cars_df3.withColumn("price_per_day", get_json_object(cars_df2.car_info, '$.price_per_day'))
cars_df5 = cars_df4.drop("car_info")

int
string


In [13]:
customers_schema = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])
customers_df = spark.read.option("header", True).schema(customers_schema).csv("/opt/spark/work-dir/data/car_service/customers")
customers_df2 = customers_df.withColumn("customer_name", get_json_object(customers_df.customer_info, '$.customer_name'))
customers_df3 = customers_df2.withColumn("city", get_json_object(customers_df2.customer_info, '$.city'))
customers_df4 = customers_df3.withColumn("age", get_json_object(customers_df2.customer_info, '$.age'))
customers_df5 = customers_df4.drop("customer_info")

int
string


In [14]:

rentals_schema = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])
rentals_df = spark.read.option("header", True).schema(rentals_schema).csv("/opt/spark/work-dir/data/car_service/rentals")
rentals_df2 = rentals_df.withColumn("car_id", get_json_object(rentals_df.rental_info, '$.car_id'))
rentals_df3 = rentals_df2.withColumn("customer_id", get_json_object(rentals_df2.rental_info, '$.customer_id'))
rentals_df4 = rentals_df3.withColumn("agency_id", get_json_object(rentals_df2.rental_info, '$.agency_id'))
rentals_df5 = rentals_df4.drop("rental_info")

int
string


### You need to generate one single dataframe using join operations and extracting columns from the json columns (using get_json_object) and using only one action (show()).


In [15]:
customer_reference = customers_df2.drop("customer_info")
car_reference = cars_df2.drop("car_info")
agency_reference = agencies_df2.drop("agency_info")

result = rentals_df5.join(car_reference, rentals_df5["car_id"] == car_reference["car_id"], "left").drop("car_id") \
        .join(customer_reference, rentals_df5["customer_id"] == customer_reference["customer_id"], "left").drop("customer_id") \
        .join(agency_reference, rentals_df5["agency_id"] == agency_reference["agency_id"], "left").drop("agency_id")
result.show()

+---------+--------------------+---------------+-------------+
|rental_id|            car_name|  customer_name|  agency_name|
+---------+--------------------+---------------+-------------+
|    11891|Wallace-Carlson M...| Margaret Jones|  NYC Rentals|
|    11892|Grimes-Green Model 8|Albert Williams|LA Car Rental|
|    11893|Stewart-Allen Mod...|  Caleb Fleming|      SF Cars|
|    11894|  Campos PLC Model 4|  Andrew Butler|  NYC Rentals|
|    11895|  Wagner LLC Model 1|  Kristin Potts|      SF Cars|
|    11896|Jones, Jefferson ...|   Jeremy Parks|LA Car Rental|
|    11897|Lopez and Sons Mo...|    Terry Wells| Zapopan Auto|
|    11898| Salazar Ltd Model 8|  Marc Williams|      SF Cars|
|    11899|Villanueva PLC Mo...| Danny Williams|LA Car Rental|
|    11900|Faulkner-Howard M...| Eric Owens PhD|      SF Cars|
|    11901|Faulkner-Howard M...|    Laura Perry|  NYC Rentals|
|    11902|Faulkner-Howard M...|     Paul Brown|  NYC Rentals|
|    11903|Atkinson Ltd Mode...|Alexa Hernandez| Zapopa

In [16]:
sc.stop()